In [28]:
# split data for target model and substitute model 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib
import seaborn as sns
import sklearn
import imblearn
import matplotlib.pyplot as plt
import time
import sklearn.metrics as m
import xgboost as xgb
# Ignore warnings
import warnings
warnings.filterwarnings('ignore')


In [29]:
#Load Data
df1=pd.read_csv('../datasets/MachineLearningCVE/CICIDS2017/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv')
df2=pd.read_csv('../datasets/MachineLearningCVE/CICIDS2017/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv')
df3=pd.read_csv('../datasets/MachineLearningCVE/CICIDS2017/Friday-WorkingHours-Morning.pcap_ISCX.csv')
df4=pd.read_csv('../datasets/MachineLearningCVE/CICIDS2017/Monday-WorkingHours.pcap_ISCX.csv') # all is benign traffic
df5=pd.read_csv('../datasets/MachineLearningCVE/CICIDS2017/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv')
df6=pd.read_csv('../datasets/MachineLearningCVE/CICIDS2017/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv')
df7=pd.read_csv('../datasets/MachineLearningCVE/CICIDS2017/Tuesday-WorkingHours.pcap_ISCX.csv')
df8=pd.read_csv('../datasets/MachineLearningCVE/CICIDS2017/Wednesday-workingHours.pcap_ISCX.csv')

In [30]:
# add a column to each dataframe to indicate if the traffic is benign or malicious
df1['Binary_class'] = df1[' Label'].map({'BENIGN': 'benign', 'DDoS': 'malicious'})
df2['Binary_class'] = df2[' Label'].map({'BENIGN': 'benign', 'PortScan': 'malicious'})
df3['Binary_class'] = df3[' Label'].map({'BENIGN': 'benign', 'Bot': 'malicious'})
df4['Binary_class'] = df4[' Label'].map({'BENIGN': 'benign'})
df5['Binary_class'] = df5[' Label'].map({'BENIGN': 'benign', 'Infiltration': 'malicious'})
df6['Binary_class'] = df6[' Label'].map({'BENIGN': 'benign', 'Web Attack � Brute Force': 'malicious', 'Web Attack � XSS': 'malicious', 'Web Attack � Sql Injection': 'malicious'})
df7['Binary_class'] = df7[' Label'].map({'BENIGN': 'benign', 'FTP-Patator': 'malicious', 'SSH-Patator': 'malicious'})
df8['Binary_class'] = df8[' Label'].map({'BENIGN': 'benign', 'DOS Hulk': 'malicious', 'DoS GoldenEye': 'malicious', 'DoS slowloris': 'malicious', 'DoS Slowhttptest': 'malicious', 'Heartbleed': 'malicious'})



#Concatenate all data into one DataFrame
df=pd.concat([df1,df2,df3, df4, df5,df6,df7,df8])
del df1,df2,df3,df4,df5,df6,df7,df8

df['attack_type'] = df[' Label'].map({'BENIGN': 'benign', 'DDoS': 'DOS', 'PortScan': 'PortScan', 'Bot': 'DoS', 'Infiltration': 'Infiltration', 
                                      'Web Attack � Brute Force': 'Web Attack', 'Web Attack � XSS': 'Web Attack', 'Web Attack � Sql Injection': 'Web Attack', 
                                      'FTP-Patator': 'Patator', 'SSH-Patator': 'Patator', 'DoS Hulk': 'DoS', 'DoS GoldenEye': 'DoS', 'DoS slowloris': 'DoS', 
                                      'DoS Slowhttptest': 'DoS', 'Heartbleed': 'Heartbleed'})

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2830743 entries, 0 to 692702
Data columns (total 81 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0    Destination Port             int64  
 1    Flow Duration                int64  
 2    Total Fwd Packets            int64  
 3    Total Backward Packets       int64  
 4   Total Length of Fwd Packets   int64  
 5    Total Length of Bwd Packets  int64  
 6    Fwd Packet Length Max        int64  
 7    Fwd Packet Length Min        int64  
 8    Fwd Packet Length Mean       float64
 9    Fwd Packet Length Std        float64
 10  Bwd Packet Length Max         int64  
 11   Bwd Packet Length Min        int64  
 12   Bwd Packet Length Mean       float64
 13   Bwd Packet Length Std        float64
 14  Flow Bytes/s                  float64
 15   Flow Packets/s               float64
 16   Flow IAT Mean                float64
 17   Flow IAT Std                 float64
 18   Flow IAT Max               

In [31]:
df.value_counts(' Label')

 Label
BENIGN                        2273097
DoS Hulk                       231073
PortScan                       158930
DDoS                           128027
DoS GoldenEye                   10293
FTP-Patator                      7938
SSH-Patator                      5897
DoS slowloris                    5796
DoS Slowhttptest                 5499
Bot                              1966
Web Attack � Brute Force         1507
Web Attack � XSS                  652
Infiltration                       36
Web Attack � Sql Injection         21
Heartbleed                         11
Name: count, dtype: int64

In [32]:
# drop duplicates
df = df.drop_duplicates()
df

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label,Binary_class,attack_type
0,54865,3,2,0,12,0,6,6,6.0,0.00000,...,0.0,0,0,0.0,0.0,0,0,BENIGN,benign,benign
1,55054,109,1,1,6,6,6,6,6.0,0.00000,...,0.0,0,0,0.0,0.0,0,0,BENIGN,benign,benign
2,55055,52,1,1,6,6,6,6,6.0,0.00000,...,0.0,0,0,0.0,0.0,0,0,BENIGN,benign,benign
3,46236,34,1,1,6,6,6,6,6.0,0.00000,...,0.0,0,0,0.0,0.0,0,0,BENIGN,benign,benign
4,54863,3,2,0,12,0,6,6,6.0,0.00000,...,0.0,0,0,0.0,0.0,0,0,BENIGN,benign,benign
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
692698,53,32215,4,2,112,152,28,28,28.0,0.00000,...,0.0,0,0,0.0,0.0,0,0,BENIGN,benign,benign
692699,53,324,2,2,84,362,42,42,42.0,0.00000,...,0.0,0,0,0.0,0.0,0,0,BENIGN,benign,benign
692700,58030,82,2,1,31,6,31,0,15.5,21.92031,...,0.0,0,0,0.0,0.0,0,0,BENIGN,benign,benign
692701,53,1048635,6,2,192,256,32,32,32.0,0.00000,...,0.0,0,0,0.0,0.0,0,0,BENIGN,benign,benign


In [21]:
from sklearn.model_selection import train_test_split

# split the benign data into training and testing
df_benign = df[df[' Label'] == 'BENIGN']
df_benign1, df_benign2 = train_test_split(df_benign, test_size=0.5, random_state=42)

df_DoS_Hulk = df[df[' Label'] == 'DoS Hulk']
df_Dos_Hulk1, df_Dos_Hulk2 = train_test_split(df_DoS_Hulk, test_size=0.5, random_state=42)

df_PortScan = df[df[' Label'] == 'PortScan']
df_PortScan1, df_PortScan2 = train_test_split(df_PortScan, test_size=0.5, random_state=42)

df_DoS_GoldenEye = df[df[' Label'] == 'DoS GoldenEye']
df_DoS_GoldenEye1, df_DoS_GoldenEye2 = train_test_split(df_DoS_GoldenEye, test_size=0.5, random_state=42)

df_DoS_slowloris = df[df[' Label'] == 'DoS slowloris']
df_DoS_slowloris1, df_DoS_slowloris2 = train_test_split(df_DoS_slowloris, test_size=0.5, random_state=42)

df_Bot = df[df[' Label'] == 'Bot']
df_Bot1, df_Bot2 = train_test_split(df_Bot, test_size=0.5, random_state=42)

df_Infiltration = df[df[' Label'] == 'Infiltration']
df_Infiltration1, df_Infiltration2 = train_test_split(df_Infiltration, test_size=0.5, random_state=42)

df_DDOS = df[df[' Label'] == 'DDoS']
df_DDOS1, df_DDOS2 = train_test_split(df_DDOS, test_size=0.5, random_state=42)

df_Heartbleed = df[df[' Label'] == 'Heartbleed']
df_Heartbleed1, df_Heartbleed2 = train_test_split(df_Heartbleed, test_size=0.5, random_state=42)

df_SSH_Patator = df[df[' Label'] == 'SSH-Patator']  
df_SSH_Patator1, df_SSH_Patator2 = train_test_split(df_SSH_Patator, test_size=0.5, random_state=42)

df_FTP_Patator = df[df[' Label'] == 'FTP-Patator']
df_FTP_Patator1, df_FTP_Patator2 = train_test_split(df_FTP_Patator, test_size=0.5, random_state=42)

df_Web_Attack_Brute_Force = df[df[' Label'] == 'Web Attack � Brute Force']
df_Web_Attack_Brute_Force1, df_Web_Attack_Brute_Force2 = train_test_split(df_Web_Attack_Brute_Force, test_size=0.5, random_state=42)    

df_Web_Attack_XSS = df[df[' Label'] == 'Web Attack � XSS']
df_Web_Attack_XSS1, df_Web_Attack_XSS2 = train_test_split(df_Web_Attack_XSS, test_size=0.5, random_state=42)

df_Web_Attack_Sql_Injection = df[df[' Label'] == 'Web Attack � Sql Injection']
df_Web_Attack_Sql_Injection1, df_Web_Attack_Sql_Injection2 = train_test_split(df_Web_Attack_Sql_Injection, test_size=0.5, random_state=42)

df_Web_Attack_Sql_Injection = df[df[' Label'] == 'Web Attack � Sql Injection']
df_Web_Attack_Sql_Injection1, df_Web_Attack_Sql_Injection2 = train_test_split(df_Web_Attack_Sql_Injection, test_size=0.5, random_state=42)

df_DoS_Slowhttptest = df[df[' Label'] == 'DoS Slowhttptest']
df_DoS_Slowhttptest1, df_DoS_Slowhttptest2 = train_test_split(df_DoS_Slowhttptest, test_size=0.5, random_state=42)

df_target = pd.concat([df_benign1, df_Dos_Hulk1, df_PortScan1, df_DoS_GoldenEye1, df_DoS_slowloris1, df_Bot1, df_Infiltration1, df_DDOS1, df_Heartbleed1, df_SSH_Patator1, df_FTP_Patator1, df_Web_Attack_Brute_Force1, df_Web_Attack_XSS1, df_Web_Attack_Sql_Injection1, df_DoS_Slowhttptest1])    
df_substitute = pd.concat([df_benign2, df_Dos_Hulk2, df_PortScan2, df_DoS_GoldenEye2, df_DoS_slowloris2, df_Bot2, df_Infiltration2, df_DDOS2, df_Heartbleed2, df_SSH_Patator2, df_FTP_Patator2, df_Web_Attack_Brute_Force2, df_Web_Attack_XSS2, df_Web_Attack_Sql_Injection2, df_DoS_Slowhttptest2])
# shuffle the data
df_target = df_target.sample(frac=1).reset_index(drop=True)
df_substitute = df_substitute.sample(frac=1).reset_index(drop=True)


In [26]:
#save the data
df_target.to_csv('../datasets/target_model/CICIDS2017/df_target_0311.csv', index=False)
df_substitute.to_csv('../datasets/surrogate_model/CICIDS2017/df_substitute_0311.csv', index=False)